In [1]:
from pathlib import Path  # pathlib: module, Path: class. Checking if a path exist
from typing import Optional, List, Dict, Tuple  # typing: support for type hint
import pandas as pd
import numpy as np
import collections  # This module contains different datatype to process the data: dict, list, set, and tuple.
from pprint import pprint  # pprint.pprint() can use when you need to examine the structure of a large or complex
# data structure. this output reveals more readable and structured way.
from who_member_states import WHO_MEMBER_STATES

In [2]:

__all__ = ['select_df']  # only import 'select_df'


# test data should <10 MB

def select_df(df: pd.DataFrame,
              rename_mapping: Dict[str, str] = None,
              column_drop: Optional[List[str]] = None,  # column_drop (param) is an optional list of string. Optional
              # type is from Typing. default is None (no drop any column). If provide column_drop, must be ['xxx'] (
              # list of string)
              year: int = 2000,
              save_path: Optional[Path] = None,
              drop_na: Optional[List[str]] = None, ) -> pd.DataFrame:
    """
    dataframe modification and save as another file

    :param df: input dataframe
    :param rename_mapping: both df have countries but the header is different
    :param column_drop: drop the column(s) that are not informative
    :param year: pick up the data that larger than which year
    :param save_path: path for saving the modified dataframe
    :param drop_na: specify if drop the na-existing column name
    :return: modified df
    """

    df = df.copy()  # The copy() method returns a copy of the DataFrame. By default, the copy is a "deep copy"
    # meaning that any changes  made in the original DataFrame will NOT be reflected in the copy. 新跑出來的df不會影響最原始的df

    if rename_mapping is not None:
        df = df.rename(columns=rename_mapping)
    if column_drop is not None:  # if drop specific column,df need to drop the option
        df = df.drop(columns=column_drop)  # df 的column = column after drop, column_drop: Optional[List[str]]

    year_mask: pd.Series[bool] = df['Year'] >= year  # type is list of bool; compared df["year"] whether larger than
    # default year (year: int = 2000)
#    entity_mask: List[bool] = [country in WHO_MEMBER_STATES for country in df['Country Name']]  # check countries whether
    # in df['Entity'] also in WHO_MEMBER_STATES. If Yes =True
    modified_df: pd.DataFrame = df[year_mask].reset_index(
        drop=True)  # create a new df that only meet both
    # year_mask and entity_mask. df.reset_index(drop = True) means new index created, old index don't added in new df.

    if drop_na is not None:  # drop rows with missing values ('NaN') from df
        try:
            modified_df.dropna(subset=drop_na, inplace=True)
            # inplace = True means that the original df will be modified and no copy will be made.; But, if inplace =
            # False, df will still show the initial one. subset = drop_na means drop in specific place you set.
        except KeyError as e:
            raise ValueError(f'{e} not in the dataframe, should be one of the {modified_df.columns.tolist()}')  # If
            # typed wrong, show the list which should be dropped.
    if save_path is not None:
        modified_df.to_excel(save_path)

    return modified_df


In [3]:
if __name__ == '__main__':
    raw_who_cvd_df = pd.read_csv('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/raw '
                                 'data/WHOMortalityDatabase_Deaths_sex_age_a_country_area_year-Cardiovascular '
                                 'diseases_7th February 2023.csv')
column_drop = ['Age group code', 'Unnamed: 12']
na_header = ['Number',
             'Percentage of cause-specific deaths out of total deaths',
             'Death rate per 100 000 population']
save_path = (
    '/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHO_Cardiovascular_Disease_Mortality_Database.xlsx')
who_cvd_df = select_df(raw_who_cvd_df, column_drop=column_drop, drop_na=na_header,
                       save_path=save_path)

In [4]:
number_of_countries=raw_who_cvd_df['Country Name'].unique()
print(len(number_of_countries),raw_who_cvd_df.shape, raw_who_cvd_df.columns.tolist())
print(len(who_cvd_df['Country Name'].unique()),who_cvd_df.shape,who_cvd_df.columns.tolist())



114 (297066, 13) ['Region Code', 'Region Name', 'Country Code', 'Country Name', 'Year', 'Sex', 'Age group code', 'Age Group', 'Number', 'Percentage of cause-specific deaths out of total deaths', 'Age-standardized death rate per 100 000 standard population', 'Death rate per 100 000 population', 'Unnamed: 12']
111 (111401, 11) ['Region Code', 'Region Name', 'Country Code', 'Country Name', 'Year', 'Sex', 'Age Group', 'Number', 'Percentage of cause-specific deaths out of total deaths', 'Age-standardized death rate per 100 000 standard population', 'Death rate per 100 000 population']


In [5]:
# 計算每列中的缺失值總數
print(raw_who_cvd_df.isna().sum(axis=0))

print(raw_who_cvd_df.isna().sum().sum())

Region Code                                                         0
Region Name                                                         0
Country Code                                                        0
Country Name                                                        0
Year                                                                0
Sex                                                                 0
Age group code                                                      0
Age Group                                                           0
Number                                                           1593
Percentage of cause-specific deaths out of total deaths          9763
Age-standardized death rate per 100 000 standard population    283341
Death rate per 100 000 population                               25020
Unnamed: 12                                                    297066
dtype: int64
616783


In [6]:
def preprocess_cvd(df: pd.DataFrame,
                   drop_na: Optional[List[str]] = None,
                   save_path: Optional[Path] = None) -> pd.DataFrame:
    """
    Dataframe of WHO_Cardiovascular_Disease_Mortality_Database.xlsx need to modify:
    - grouping if set kwarg `grouping_age` as true  # kwarg : keyword arguments
    - Calculate Total number of death
    - only left age >15 years old
    - save as another dataframe

    :param df: input dataframe (WHO_Cardiovascular_Disease_Mortality_Database.xlsx)
    :param drop_na: drop Age_Group <15 year-old
    :param save_path: save modified dataframe to another excel
    :return: df
    """
    
    # Sum of number of death in each age group
    numbers = df['Number']
    percentage = df['Percentage of cause-specific deaths out of total deaths']
    df["Total Number of Cause-Specific Deaths"] = numbers * 100 / percentage
    total_number_of_death = df["Total Number of Cause-Specific Deaths"]
    mask_nan = np.isnan(total_number_of_death)  # type: pd.Series[bool] # if value is NaN, NaN = True

    df.loc[mask_nan, "Total Number of Cause-Specific Deaths"] = 0  # search location of df. if index ('Total number of death') is
    # NaN, change NaN to 0. ( if no.loc :SettingWithCopyWarning: A value is trying to be set on a copy of a slice
    # from a DataFrame
    df["Total Number of Cause-Specific Deaths"] = df["Total Number of Cause-Specific Deaths"].astype(int)  # astype can cast/change multiple types (
    # change type to int)
    if drop_na is not None:  # drop rows with missing values ('NaN') from df
        df = df.mask(df['Age Group'].isin(['[0]', '[1-4]', '[5-9]', '[10-14]']), np.nan)
        df.dropna(subset=['Age Group'], inplace=True)
    
    if save_path is not None:
        df.to_excel(save_path)
    return df


In [7]:
who_cvd_df= pd.read_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHO_Cardiovascular_Disease_Mortality_Database.xlsx')
# “xlrd” supports old-style Excel files (.xls).“openpyxl” supports newer Excel file formats.
drop_na =['Age Group']
save_path=('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHO_Cardiovascular_Disease_Mortality_Database_preprocess.xlsx')
who_cvd_df_preprocess = preprocess_cvd(who_cvd_df,drop_na= drop_na,save_path=save_path)  # assign a who_cvd_df after preprocess_cvd
print(len(who_cvd_df_preprocess['Country Name'].unique()),who_cvd_df_preprocess.shape,who_cvd_df_preprocess.columns.tolist())

111 (89218, 13) ['Unnamed: 0', 'Region Code', 'Region Name', 'Country Code', 'Country Name', 'Year', 'Sex', 'Age Group', 'Number', 'Percentage of cause-specific deaths out of total deaths', 'Age-standardized death rate per 100 000 standard population', 'Death rate per 100 000 population', 'Total Number of Cause-Specific Deaths']


In [8]:
def create_age_grouping(df: pd.DataFrame,
                        save_path: Optional[Path] = None) -> pd.DataFrame:
    """
    Calculate: Total percentage of CVD of total deaths = Sum of number/ Sum of Total number of cause-specific deaths
    * 100 (Male/ Female/ All in each year and country)
    grouping_age: Age groups --> one age group (greater 15 y/o)
    create a new df and save it to excel

    :param df: df after select_df and preprocess_cvd
    :param save: save modified dataframe to another excel
    :return: new df
    """

    if 'Total Number of Cause-Specific Deaths' not in df.columns:
        raise RuntimeError('call preprocess_cvd in advance')

    dy: dict = collections.defaultdict(list)  # defaultdict object in collections. datatype will be dict. Using list
    # as the default_factory to group a sequence of key-value pairs into a dictionary of lists
    group = df.groupby(['Region Code','Region Name','Country Code','Country Name', 'Year', 'Sex'])
    info: List[Tuple] = list(group.groups.keys())  # List[Tuple]: value is a list of tuple[()].looking for the keys in a
    # dict. The 'groups' attribute of the 'groupby' object is always dic type

    for i, it in enumerate(info):  # i = index ( starting from 0) , it = item (Entity, Year, Sex). enumerate can pair
        # index and item
        dy['Region Code'].append(it[0])
        dy['Region Name'].append(it[1]) 
        dy['Country Code'].append(it[2]) 
        dy['Country Name'].append(it[3])  
        dy['Year'].append(it[4])  # Year in [4]
        dy['Sex'].append(it[5])  # Sex in [5]

    numbers = group['Number']
    total_number_of_death = group['Total Number of Cause-Specific Deaths']
    # noinspection PyTypeChecker todo: wt is it?
    dy['Number'] = np.array(numbers.sum())
    dy['Total Number of Cause-Specific Deaths'] = np.array(total_number_of_death .sum())
    dy['Total Percentage of Cause-Specific Deaths Out Of Total Deaths'] = np.array(numbers.sum() / total_number_of_death.sum() * 100)

    _df = pd.DataFrame.from_dict(dy)  # creates a new_df from the dy dictionary.

    # change layout
    sex_values = ['All', 'Female', 'Male']
    new_df = _df.assign(
        All_Number=_df.query("Sex == 'All'")['Number'],
        Female_Number=_df.query("Sex == 'Female'")['Number'],
        Male_Number=_df.query("Sex == 'Male'")['Number'],
        All_Total_Number_of_Cause_Specific_Deaths=_df.query("Sex == 'All'")['Total Number of Cause-Specific Deaths'],
        Female_Total_Number_of_Cause_Specific_Deaths=_df.query("Sex == 'Female'")['Total Number of Cause-Specific Deaths'],
        Male_Total_Number_of_Cause_Specific_Deaths=_df.query("Sex == 'Male'")['Total Number of Cause-Specific Deaths'],
        All_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths=_df.query("Sex == 'All'")['Total Percentage of Cause-Specific Deaths Out Of Total Deaths'],
        Female_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths=_df.query("Sex == 'Female'")['Total Percentage of Cause-Specific Deaths Out Of Total Deaths'],
        Male_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths=_df.query("Sex == 'Male'")['Total Percentage of Cause-Specific Deaths Out Of Total Deaths'])
    new_df.reset_index(drop=True, inplace=True)

    new_df = new_df.drop(['Sex', 'Number', 'Total Number of Cause-Specific Deaths', 'Total Percentage of Cause-Specific Deaths Out Of Total Deaths'],
                         axis=1)  # axis = 1: specifies to drop columns

    new_df = new_df.groupby(['Region Code','Region Name','Country Code','Country Name', 'Year']).first().reset_index()  # The first method is then applied to
    # the grouped dataframe, which returns the first row of each group
    if save_path:
        new_df.to_excel(save_path)
    return new_df


In [9]:
who_cvd_df_preprocess = pd.read_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHO_Cardiovascular_Disease_Mortality_Database_preprocess.xlsx')
who_cvd_df_preprocess= preprocess_cvd(who_cvd_df_preprocess)
save_path = ('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/new_WHO_Cardiovascular_Disease_Mortality_Database.xlsx')
new_df = create_age_grouping(who_cvd_df_preprocess,save_path=save_path)
print(len(new_df['Country Name'].unique()),new_df.shape,new_df.columns.tolist())

111 (1878, 14) ['Region Code', 'Region Name', 'Country Code', 'Country Name', 'Year', 'All_Number', 'Female_Number', 'Male_Number', 'All_Total_Number_of_Cause_Specific_Deaths', 'Female_Total_Number_of_Cause_Specific_Deaths', 'Male_Total_Number_of_Cause_Specific_Deaths', 'All_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths', 'Female_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths', 'Male_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths']


In [10]:
def tobacco_layout_modified(df: pd.DataFrame,
                            column_drop: Optional[List[str]] =None,
                            save_path: Optional[Path] = None) -> pd.DataFrame:
    if column_drop is not None:
        df = df.drop(columns=column_drop)
    dy: dict = collections.defaultdict(list)
    group = df.groupby(['Country Name', 'Year', 'Indicator', 'Sex'])
    info: List[Tuple] = list(group.groups.keys())

    for i, it in enumerate(info):
        dy['Country Name'].append(it[0])
        dy['Year'].append(it[1])
        dy['Indicator'].append(it[2])
        dy['Sex'].append(it[3])
        dy['Prevalence'].append(group.get_group(it).Prevalence.mean())

    _df = pd.DataFrame.from_dict(dy)
    sex_values = ['Both sexes', 'Male', 'Female']
    indicator_values = ['Estimate of current tobacco use prevalence (%) (age-standardized rate)',
                        'Estimate of current tobacco smoking prevalence (%) (age-standardized rate)']

    changed_df = _df.assign(
        All_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate=_df.query(
            "Sex == 'Both sexes'& Indicator =='Estimate of current tobacco use prevalence (%) (age-standardized rate)'")[
            'Prevalence'],
        Male_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate=_df.query(
            "Sex == 'Male'& Indicator =='Estimate of current tobacco use prevalence (%) (age-standardized rate)'")[
            'Prevalence'],
        Female_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate=_df.query(
            "Sex == 'Female'& Indicator =='Estimate of current tobacco use prevalence (%) (age-standardized rate)'")[
            'Prevalence'],

        All_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate=_df.query(
            "Sex == 'Both sexes'& Indicator =='Estimate of current tobacco smoking prevalence (%) (age-standardized "
            "rate)'")['Prevalence'],
        Male_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate=_df.query(
            "Sex == 'Male'& Indicator =='Estimate of current tobacco smoking prevalence (%) (age-standardized rate)'")[
            'Prevalence'],
        Female_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate=_df.query(
            "Sex == 'Female'& Indicator =='Estimate of current tobacco smoking prevalence (%) (age-standardized rate)'")[
            'Prevalence'])

    changed_df.reset_index(drop=True, inplace=True)

    changed_df = changed_df.drop(['Sex', 'Prevalence', 'Indicator'], axis=1)
    changed_df = changed_df.groupby(['Country Name', 'Year']).first().reset_index()
    print(changed_df)
    if save_path is not None:
        changed_df.to_excel(save_path)
    return changed_df


In [11]:
raw_tobacco_df = pd.read_csv(
    '/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/raw data/Estimate of current tobacco smoking '
    'prevalence(%)(age-standardized rate)_17 Jan 2022.csv')

rename = {'Location': 'Country Name', 'Period': 'Year', 'Dim1': 'Sex', 'First Tooltip': 'Prevalence'}

save_path = '/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/Prevalence of Tobacco use_modified.xlsx'
tobacco_df = select_df(raw_tobacco_df, rename_mapping=rename, save_path=save_path)
print(len(raw_tobacco_df['Location'].unique()),raw_tobacco_df.shape,raw_tobacco_df.columns.tolist())
print(len(tobacco_df['Country Name'].unique()),tobacco_df.shape,tobacco_df.columns.tolist())

165 (9351, 5) ['Location', 'Period', 'Indicator', 'Dim1', 'First Tooltip']
165 (9351, 5) ['Country Name', 'Year', 'Indicator', 'Sex', 'Prevalence']


In [12]:
print(raw_tobacco_df.isna().sum(axis=0))

print(raw_tobacco_df.isna().sum().sum())

Location         0
Period           0
Indicator        0
Dim1             0
First Tooltip    0
dtype: int64
0


In [13]:
tobacco_df = pd.read_excel(
    '/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/Prevalence of Tobacco use_modified.xlsx')

save_path = '/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/Prevalence of Tobacco use_Changed_layout.xlsx'
changed_df = tobacco_layout_modified(tobacco_df, save_path=save_path)

print(len(changed_df['Country Name'].unique()),changed_df.shape,changed_df.columns.tolist())


     Country Name  Year  \
0     Afghanistan  2000   
1     Afghanistan  2005   
2     Afghanistan  2010   
3     Afghanistan  2015   
4     Afghanistan  2018   
...           ...   ...   
1472     Zimbabwe  2018   
1473     Zimbabwe  2019   
1474     Zimbabwe  2020   
1475     Zimbabwe  2023   
1476     Zimbabwe  2025   

      All_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate  \
0                                                  36.9                      
1                                                  32.7                      
2                                                  29.1                      
3                                                  26.0                      
4                                                  23.8                      
...                                                 ...                      
1472                                               12.1                      
1473                                               

In [14]:
# merge df1 & df2 test
df1 = new_df
df2 = changed_df

cvd_tobacco = pd.merge(df1, df2, on=['Country Name', 'Year'], how='outer')
cvd_tobacco.fillna(value='NaN',
                   inplace=True)  # inplace = True means that 'value = 'NaN'' will inplace original value in df. 'Nan' can changed what you want to instead of.
cvd_tobacco.to_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/merge_cvd_tobacco.xlsx')

print(len(cvd_tobacco['Country Name'].unique()),cvd_tobacco.shape, cvd_tobacco.columns.tolist())


185 (2895, 20) ['Region Code', 'Region Name', 'Country Code', 'Country Name', 'Year', 'All_Number', 'Female_Number', 'Male_Number', 'All_Total_Number_of_Cause_Specific_Deaths', 'Female_Total_Number_of_Cause_Specific_Deaths', 'Male_Total_Number_of_Cause_Specific_Deaths', 'All_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths', 'Female_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths', 'Male_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths', 'All_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate', 'Male_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate', 'Female_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate', 'All_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate', 'Male_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate', 'Female_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate']


In [15]:
cvd_tobacco_nomissing = pd.read_excel(
    '/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/merge_cvd_tobacco_no_missingdata.xlsx')
print(len(cvd_tobacco_nomissing['Country Name'].unique()))

89


In [16]:
print(cvd_tobacco.eq('NaN').sum().sum())

20718


In [17]:
def format_date(df: pd.DataFrame,
                rename_mapping: Dict[str, str] = None,
                formatted_date: Optional[List[str]] = None,
                save_path: Optional[Path] = None) -> pd.DataFrame:
    if rename_mapping is not None:
        df = df.rename(columns=rename_mapping)
    if formatted_date is not None:
        for col in formatted_date:
            df[col] = pd.to_datetime(df[col], infer_datetime_format=True)
            df[col] = df[col].dt.strftime("%d %b %Y")
    df.fillna(value='Nan', inplace=True)
    if save_path is not None:
        df.to_excel(save_path)
    return df

In [18]:
if __name__ == '__main__':
    df = pd.read_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/raw data/Signatures and '
                       'Ratifications- UN Treaty Section_08 Feb_2023 .xlsx', engine='openpyxl')
    rename = {'Participant': 'Country Name',
              "Ratification, Acceptance(A), Approval(AA), Formal confirmation(c), Accession(a), Succession(d)": 'Ratification'}
    formatted_date = ['Signature', 'Ratification']
    save_path = '/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHOFCTC_Parties_date_formatted.xlsx'
    df=format_date(df, rename_mapping=rename, formatted_date=formatted_date, save_path=save_path)
    


print(len(df['Country Name'].unique()),df.shape,df.columns.tolist())

188 (188, 3) ['Country Name', 'Signature', 'Ratification']


In [19]:
# merge
df1 = pd.read_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/merge_cvd_tobacco.xlsx',
                    engine='openpyxl')
df2 = pd.read_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHOFCTC_Parties_date_formatted.xlsx',
                    engine='openpyxl')

signed_df = pd.merge(df1, df2, on=['Country Name'], how='outer')
signed_df.fillna(value='NaN', inplace=True)
signed_df.to_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHOFCTC_Parties_signed_date.xlsx')

In [20]:
print(signed_df.eq('NaN').sum().sum())

22457


In [21]:

# drop_missing_data
df = pd.read_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHOFCTC_Parties_signed_date.xlsx')
df = df.dropna(how='any')  # drop the rows having Nan
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0_x', 'Unnamed: 0_y'])

df.to_excel(
    '/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHOFCTC_Parties_signed_date_no_missingdata.xlsx')


countries = df['Country Name'].unique()
number_of_countries = len(countries)
print(number_of_countries, countries, df.shape)  


82 ['Egypt' 'Mauritius' 'Seychelles' 'South Africa' 'Armenia' 'Azerbaijan'
 'Bahrain' 'Brunei Darussalam' 'Cyprus' 'Georgia'
 'Iran (Islamic Republic of)' 'Iraq' 'Israel' 'Jordan' 'Japan'
 'Kazakhstan' 'Kyrgyzstan' 'Republic of Korea' 'Kuwait' 'Lebanon'
 'Sri Lanka' 'Maldives' 'Philippines' 'Singapore' 'Thailand'
 'Turkmenistan' 'Uzbekistan' 'Argentina' 'Belize' 'Brazil' 'Chile'
 'Colombia' 'Costa Rica' 'Ecuador' 'Guatemala' 'Guyana' 'Mexico' 'Panama'
 'Peru' 'Paraguay' 'El Salvador' 'Albania' 'Austria' 'Belgium' 'Bulgaria'
 'Bosnia and Herzegovina' 'Belarus' 'Switzerland' 'Germany' 'Spain'
 'Estonia' 'Finland' 'France' 'Greece' 'Croatia' 'Hungary' 'Ireland'
 'Iceland' 'Italy' 'Lithuania' 'Luxembourg' 'Latvia' 'Republic of Moldova'
 'Malta' 'Netherlands' 'Norway' 'Poland' 'Portugal' 'Romania'
 'Russian Federation' 'Serbia' 'Slovakia' 'Slovenia' 'Sweden' 'Ukraine'
 'Bahamas' 'Barbados' 'Canada' 'Cuba' 'Jamaica' 'United States of America'
 'Fiji'] (424, 22)


In [22]:
#mask non ratified parties

mask = df['Ratification'] != 'Nan'
df = df[mask]
df = df[df['Ratification'] != 'Nan']
df.to_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH '
            'Dissertation/Data/WHOFCTC_Parties_ratified_date_no_missingdata.xlsx') 

print(df['Country Name'].unique(),len(df['Country Name'].unique()), df.shape,df.columns.tolist())  


['Egypt' 'Mauritius' 'Seychelles' 'South Africa' 'Armenia' 'Azerbaijan'
 'Bahrain' 'Brunei Darussalam' 'Cyprus' 'Georgia'
 'Iran (Islamic Republic of)' 'Iraq' 'Israel' 'Jordan' 'Japan'
 'Kazakhstan' 'Kyrgyzstan' 'Republic of Korea' 'Kuwait' 'Lebanon'
 'Sri Lanka' 'Maldives' 'Philippines' 'Singapore' 'Thailand'
 'Turkmenistan' 'Uzbekistan' 'Belize' 'Brazil' 'Chile' 'Colombia'
 'Costa Rica' 'Ecuador' 'Guatemala' 'Guyana' 'Mexico' 'Panama' 'Peru'
 'Paraguay' 'El Salvador' 'Albania' 'Austria' 'Belgium' 'Bulgaria'
 'Bosnia and Herzegovina' 'Belarus' 'Germany' 'Spain' 'Estonia' 'Finland'
 'France' 'Greece' 'Croatia' 'Hungary' 'Ireland' 'Iceland' 'Italy'
 'Lithuania' 'Luxembourg' 'Latvia' 'Republic of Moldova' 'Malta'
 'Netherlands' 'Norway' 'Poland' 'Portugal' 'Romania' 'Russian Federation'
 'Serbia' 'Slovakia' 'Slovenia' 'Sweden' 'Ukraine' 'Bahamas' 'Barbados'
 'Canada' 'Jamaica' 'Fiji'] 78 (400, 22) ['Region Code', 'Region Name', 'Country Code', 'Country Name', 'Year', 'All_Number', 'Femal

In [23]:
print(df.eq('Nan').sum().sum())

36


In [38]:
count_country= df['Country Name'].value_counts()
count_country.to_excel('/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/count_country.xlsx')

In [50]:
df[df['Country Name']=='Slovenia']

,Region Code,Region Name,Country Code,Country Name,Year,All_Number,Female_Number,Male_Number,All_Total_Number_of_Cause_Specific_Deaths,Female_Total_Number_of_Cause_Specific_Deaths,...,Female_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths,Male_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths,All_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate,Male_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate,Female_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate,All_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate,Male_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate,Female_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate,Signature,Ratification
1823,EU,Europe,SVN,Slovenia,2000.0,15007.0,8218.0,6789.0,37035.0,17975.0,...,45.719054,35.678999,25.1,27.0,23.1,25.1,27.0,23.1,25 Sep 2003,15 Mar 2005
1828,EU,Europe,SVN,Slovenia,2005.0,15290.0,8703.0,6587.0,37520.0,18735.0,...,46.453163,35.117556,24.2,26.2,22.1,24.2,26.2,22.1,25 Sep 2003,15 Mar 2005
1833,EU,Europe,SVN,Slovenia,2010.0,14661.0,8519.0,6142.0,37120.0,18543.0,...,45.941865,33.252125,23.5,25.7,21.3,23.5,25.7,21.3,25 Sep 2003,15 Mar 2005
1838,EU,Europe,SVN,Slovenia,2015.0,15963.0,9595.0,6368.0,39597.0,20157.0,...,47.601330,32.982856,22.7,25.0,20.4,22.7,25.0,20.4,25 Sep 2003,15 Mar 2005
1841,EU,Europe,SVN,Slovenia,2018.0,15720.0,9125.0,6595.0,40873.0,20706.0,...,44.069352,32.804417,22.3,24.7,20.0,22.4,24.7,20.0,25 Sep 2003,15 Mar 2005
1842,EU,Europe,SVN,Slovenia,2019.0,15773.0,9442.0,6331.0,41079.0,20869.0,...,45.244142,31.403770,22.3,24.7,19.8,22.3,24.7,19.8,25 Sep 2003,15 Mar 2005
1843,EU,Europe,SVN,Slovenia,2020.0,15428.0,9162.0,6266.0,47952.0,24501.0,...,37.394392,26.758338,22.0,24.4,19.6,22.0,24.4,19.6,25 Sep 2003,15 Mar 2005


In [51]:
df[df['Country Name']=='Georgia']

,Region Code,Region Name,Country Code,Country Name,Year,All_Number,Female_Number,Male_Number,All_Total_Number_of_Cause_Specific_Deaths,Female_Total_Number_of_Cause_Specific_Deaths,...,Female_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths,Male_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths,All_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate,Male_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate,Female_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate,All_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate,Male_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate,Female_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate,Signature,Ratification
223,AS,Asia,GEO,Georgia,2000.0,59358.0,32178.0,27180.0,81749.0,41620.0,...,77.313791,67.751826,32.8,60.2,5.4,32.8,60.2,5.4,20 Feb 2004,14 Feb 2006
226,AS,Asia,GEO,Georgia,2005.0,54883.0,28039.0,26844.0,80178.0,38633.0,...,72.577848,64.629830,32.6,59.3,5.8,32.6,59.3,5.8,20 Feb 2004,14 Feb 2006
230,AS,Asia,GEO,Georgia,2010.0,28776.0,14764.0,14012.0,94634.0,45933.0,...,32.142468,28.776210,32.2,58.1,6.2,32.2,58.1,6.2,20 Feb 2004,14 Feb 2006
235,AS,Asia,GEO,Georgia,2015.0,41849.0,22615.0,19234.0,97500.0,47523.0,...,47.587484,38.497258,32.0,57.3,6.6,32.0,57.3,6.6,20 Feb 2004,14 Feb 2006
238,AS,Asia,GEO,Georgia,2018.0,43082.0,22556.0,20526.0,92433.0,45119.0,...,49.992243,43.385259,31.7,56.7,6.9,31.8,56.7,6.9,20 Feb 2004,14 Feb 2006
239,AS,Asia,GEO,Georgia,2019.0,42892.0,22413.0,20479.0,92769.0,45043.0,...,49.759119,42.916719,31.7,56.3,7.0,31.7,56.3,7.0,20 Feb 2004,14 Feb 2006
240,AS,Asia,GEO,Georgia,2020.0,43786.0,22723.0,21063.0,100545.0,48417.0,...,46.931863,40.413285,31.7,56.3,7.1,31.7,56.3,7.1,20 Feb 2004,14 Feb 2006


In [52]:
df[df['Country Name']=='Guatemala']

,Region Code,Region Name,Country Code,Country Name,Year,All_Number,Female_Number,Male_Number,All_Total_Number_of_Cause_Specific_Deaths,Female_Total_Number_of_Cause_Specific_Deaths,...,Female_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths,Male_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths,All_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate,Male_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate,Female_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate,All_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate,Male_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate,Female_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate,Signature,Ratification
790,CSA,Central and South America,GTM,Guatemala,2000.0,14019.0,7073.0,6946.0,113226.0,47925.0,...,14.758477,10.637386,14.3,25.2,3.3,14.3,25.2,3.3,25 Sep 2003,16 Nov 2005
795,CSA,Central and South America,GTM,Guatemala,2005.0,17225.0,8642.0,8583.0,125275.0,51997.0,...,16.620190,11.717566,13.4,23.9,2.8,13.4,23.9,2.8,25 Sep 2003,16 Nov 2005
800,CSA,Central and South America,GTM,Guatemala,2010.0,20240.0,9933.0,10307.0,131974.0,55993.0,...,17.739717,13.567019,12.5,22.6,2.3,12.5,22.6,2.3,25 Sep 2003,16 Nov 2005
805,CSA,Central and South America,GTM,Guatemala,2015.0,25447.0,12677.0,12770.0,148906.0,65069.0,...,19.482396,15.233210,11.6,21.2,1.9,11.6,21.2,1.9,25 Sep 2003,16 Nov 2005
808,CSA,Central and South America,GTM,Guatemala,2018.0,25891.0,12748.0,13143.0,154603.0,69465.0,...,18.351688,15.438197,11.0,20.5,1.7,11.1,20.5,1.7,25 Sep 2003,16 Nov 2005
809,CSA,Central and South America,GTM,Guatemala,2019.0,26912.0,13388.0,13524.0,159901.0,72128.0,...,18.561446,15.409683,11.0,20.4,1.6,11.0,20.4,1.6,25 Sep 2003,16 Nov 2005
810,CSA,Central and South America,GTM,Guatemala,2020.0,32753.0,14934.0,17819.0,183794.0,78038.0,...,19.136831,16.850756,10.9,20.1,1.6,10.9,20.1,1.6,25 Sep 2003,16 Nov 2005


In [53]:
df[df['Country Name']=='Serbia']

,Region Code,Region Name,Country Code,Country Name,Year,All_Number,Female_Number,Male_Number,All_Total_Number_of_Cause_Specific_Deaths,Female_Total_Number_of_Cause_Specific_Deaths,...,Female_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths,Male_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths,All_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate,Male_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate,Female_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate,All_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate,Male_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate,Female_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate,Signature,Ratification
1778,EU,Europe,SRB,Serbia,2000.0,116156.0,61499.0,54657.0,206824.0,100076.0,...,61.452296,51.208132,45.1,51.0,39.1,45.1,51.0,39.1,28 Jun 2004,08 Feb 2006
1783,EU,Europe,SRB,Serbia,2005.0,121331.0,65410.0,55921.0,212643.0,104522.0,...,62.580127,51.723628,43.7,48.0,39.3,43.7,48.0,39.3,28 Jun 2004,08 Feb 2006
1788,EU,Europe,SRB,Serbia,2010.0,112872.0,61620.0,51252.0,205718.0,102145.0,...,60.326007,49.490629,42.3,45.4,39.1,42.3,45.4,39.1,28 Jun 2004,08 Feb 2006
1793,EU,Europe,SRB,Serbia,2015.0,108736.0,59365.0,49371.0,206814.0,102830.0,...,57.731207,47.482160,40.9,42.7,39.1,40.9,42.7,39.1,28 Jun 2004,08 Feb 2006
1796,EU,Europe,SRB,Serbia,2018.0,105311.0,56956.0,48355.0,202786.0,101096.0,...,56.338530,47.557461,40.1,41.4,39.0,40.2,41.4,39.0,28 Jun 2004,08 Feb 2006
1797,EU,Europe,SRB,Serbia,2019.0,97547.0,53043.0,44504.0,202420.0,100092.0,...,52.994245,43.494918,40.1,41.1,39.0,40.1,41.1,39.0,28 Jun 2004,08 Feb 2006
1798,EU,Europe,SRB,Serbia,2020.0,102086.0,55427.0,46659.0,233181.0,112595.0,...,49.226875,38.695793,39.8,40.5,39.1,39.8,40.5,39.1,28 Jun 2004,08 Feb 2006


In [63]:
df[df['Country Name']=='Croatia']

,Region Code,Region Name,Country Code,Country Name,Year,All_Number,Female_Number,Male_Number,All_Total_Number_of_Cause_Specific_Deaths,Female_Total_Number_of_Cause_Specific_Deaths,...,Female_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths,Male_Total_Percentage_of_Cause_Specific_Deaths_Out_Of_Total_Deaths,All_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate,Male_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate,Female_Estimate_of_Current_Tobacco_Use_Prevalence_age_standardized_rate,All_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate,Male_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate,Female_Estimate_of_Current_Tobacco_Smoking_Prevalence_age_standardized_rate,Signature,Ratification
1377,EU,Europe,HRV,Croatia,2000.0,53414.0,29904.0,23510.0,99987.0,49320.0,...,60.632603,46.408338,34.4,41.7,27.1,34.4,41.7,27.1,02 Jun 2004,14 Jul 2008
1382,EU,Europe,HRV,Croatia,2005.0,52052.0,29370.0,22682.0,103214.0,51283.0,...,57.270440,43.683917,34.9,40.5,29.2,34.9,40.5,29.2,02 Jun 2004,14 Jul 2008
1387,EU,Europe,HRV,Croatia,2010.0,51254.0,29400.0,21854.0,103899.0,52695.0,...,55.792770,42.688596,35.4,39.5,31.3,35.4,39.5,31.3,02 Jun 2004,14 Jul 2008
1392,EU,Europe,HRV,Croatia,2015.0,51386.0,29374.0,22012.0,108160.0,55424.0,...,52.998701,41.777221,36.1,38.5,33.7,36.1,38.5,33.7,02 Jun 2004,14 Jul 2008
1395,EU,Europe,HRV,Croatia,2019.0,43898.0,25282.0,18616.0,103382.0,52764.0,...,47.915245,36.802151,36.7,37.8,35.6,36.7,37.8,35.6,02 Jun 2004,14 Jul 2008


In [ ]:

import matplotlib.pyplot as plt

WHOFCTC_Parties_date = pd.read_excel("/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/WHOFCTC_Parties_date_filter.xlsx")
CVD_Tobacco_ratified_parites = pd.read_excel("/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/CVD_Tobacco_Parties_ratified.xlsx")
WHOFCTC_Parties_date = WHOFCTC_Parties_date[WHOFCTC_Parties_date['Ratification'] != 'Nan']
WHOFCTC_Parties_date['Ratification'] = pd.to_datetime(WHOFCTC_Parties_date['Ratification'], infer_datetime_format='%d/%m/%Y')
WHOFCTC_Parties_date['Year'] = WHOFCTC_Parties_date['Ratification'].dt.year
WHOFCTC_Parties_date

In [ ]:
CVD_Tobacco_ratified_parites = CVD_Tobacco_ratified_parites[CVD_Tobacco_ratified_parites['Country'] == 'Australia']
years = CVD_Tobacco_ratified_parites['Year'].astype(str)
plt.plot(years, CVD_Tobacco_ratified_parites['Male_total_percentage_of_CVD'], label='Total Percentage of CVD in Males')
plt.plot(years, CVD_Tobacco_ratified_parites['Prevalence of current tobacco use, males (% of male adults)'], label='Prevalence of Current Tobacco Use in Males')
plt.xlabel('Year')
plt.ylabel('Percentage/Prevalence')
plt.legend()
plt.xticks(years, rotation=90)
plt.title('CVD and Tobacco Use in Australia')
plt.show()


In [ ]:

def plot_relationship(df: pd.DataFrame,
                      select_country: Optional[List[str]] = None,
                      variable_1: Optional[List[str]] = None,
                      variable_2: Optional[List[str]] = None,
                      x_label: str = None,
                      y_label: str = None,
                      save_path: Optional[Path] = None) -> pd.DataFrame:
    df['Year'] = df['Year'].astype(str)
    if select_country is not None:
        for country in select_country:
            df = df[df['Entity'] == country]
        #df = df[df['Entity'].isin(select_country)]
            if variable_1 is not None:
                plt.plot(df['Year'], df[variable_1[0]], label=variable_1[0])
            if variable_2 is not None:
                plt.plot(df['Year'], df[variable_2[0]], label=variable_2[0])
            if x_label is not None:
                plt.xlabel(x_label)
            if y_label is not None:
                plt.ylabel(y_label)
            plt.legend()
            plt.xticks(df['Year'], rotation=90)

            plt.title(f'CVD Mortality and Prevalence of Tobacco Use in {select_country[0]}')
            plt.show()

    if save_path is not None:
        df.to_excel(save_path)
    return df



In [ ]:
def plot_relationship(df: pd.DataFrame,
                      select_country: Optional[List[str]] = None,
                      variable_1: Optional[List[str]] = None,
                      variable_2: Optional[List[str]] = None,
                      x_label: str = None,
                      y_label: str = None,
                      save_path: Optional[Path] = None) -> pd.DataFrame:
    df['Year'] = df['Year'].astype(str)
    if select_country is not None:
        for country in select_country:
            country_df = df[df['Entity'] == country]
            if variable_1 is not None:
                plt.plot(country_df['Year'], country_df[variable_1[0]], label=variable_1[0])
            if variable_2 is not None:
                plt.plot(country_df['Year'], country_df[variable_2[0]], label=variable_2[0])
            if x_label is not None:
                plt.xlabel(x_label)
            if y_label is not None:
                plt.ylabel(y_label)
            plt.legend()
            plt.xticks(country_df['Year'], rotation=90)

            plt.title(f'CVD Mortality and Prevalence of Tobacco Use in {country}')
            plt.show()

    if save_path is not None:
        df.to_excel(save_path)
    return df
if __name__ == '__main__':
    df= pd.read_excel(
        "/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/CVD_Tobacco_Parties_ratified.xlsx")
    select_country =df['Entity'].unique()
    variable_1 = ['Female_total_percentage_of_CVD']
    variable_2 = ['Prevalence of current tobacco use, females (% of female adults)']
    x_label = 'Year'
    y_label = 'Total Percentage of CVD Mortality & Prevalence of Tobacco Use'
    for country in select_country:
        df = plot_relationship(df, select_country=[country], variable_1=variable_1, variable_2=variable_2,
                           x_label=x_label, y_label=y_label)


In [ ]:
df= pd.read_excel(
        "/Users/wei/UCD-MPH/MPH-Lecture:Modules/MPH Dissertation/Data/merge_cvd_tobacco_no_missingdata.xlsx")
print(df['Country Name'].unique(),len(df['Country Name'].unique()), df.shape,df.columns.tolist())  